# Module 7B: Advanced PySpark Streaming Patterns
*Stream Joins, Sliding Windows, Stateful Processing, and Alternative Data Sources*

## Learning Objectives
Master advanced streaming patterns and alternative approaches:

**Advanced Windowing**
- Sliding windows with overlapping time periods
- Session windows for user activity tracking
- Custom window functions and complex aggregations
- Multi-level windowing strategies

**Stream Joins**
- Stream-to-stream joins for real-time correlation
- Stream-to-static joins with reference data
- Temporal joins with time-based conditions
- Join strategies and optimization techniques

**Stateful Stream Processing**
- Custom stateful transformations with mapGroupsWithState
- Managing state lifecycle and memory usage
- Arbitrary stateful operations for complex use cases
- State store management and recovery

**Alternative Data Sources**
- Rate sources for testing and benchmarking
- Socket sources for real-time TCP data ingestion
- Memory sources for development and debugging
- Custom source implementations

**Advanced Output Patterns**
- File sinks with partitioning strategies
- Memory sinks for testing and validation
- Foreach sinks for custom output logic
- Delta Lake integration for ACID streaming

---

## Module Structure
1. **Environment Setup** - Advanced streaming configuration
2. **Sliding Window Analytics** - Overlapping time-based analysis
3. **Stream-to-Stream Joins** - Real-time data correlation
4. **Alternative Data Sources** - Rate, socket, and memory streaming
5. **Stateful Processing** - Custom state management
6. **Advanced Output Sinks** - File and custom outputs

In [5]:
# Module 7B: Advanced Streaming Environment Setup
print("Setting up Advanced PySpark Streaming Environment...")

import os
import time
import random
import json
import socket
import threading
import builtins  # Fix function name conflicts
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.streaming import StreamingQuery

# Explicit imports for commonly used functions to avoid issues
from pyspark.sql.functions import col, window, count, sum, avg, max, min, countDistinct, round, rand, when, lit, concat, approx_count_distinct, expr, to_date, regexp_extract, collect_list, last, unix_timestamp

# Configure Spark for Advanced Streaming workloads
spark = SparkSession.builder \
    .appName("PySpark-Advanced-Streaming-Patterns") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/advanced-streaming-checkpoints") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .config("spark.default.parallelism", "4") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.sql.streaming.stateStore.stateSchemaCheck", "false") \
    .getOrCreate()

# Set log level to reduce noise
spark.sparkContext.setLogLevel("ERROR")  # Suppress warnings for cleaner output

print("Advanced Streaming Session Created")
print("Spark Version: {}".format(spark.version))
print("Advanced checkpoint location: /tmp/advanced-streaming-checkpoints")

# Create output directories for advanced patterns
output_dir = "/tmp/advanced_streaming_output"
parquet_dir = f"{output_dir}/parquet_sink"
json_dir = f"{output_dir}/json_sink"
os.makedirs(parquet_dir, exist_ok=True)
os.makedirs(json_dir, exist_ok=True)

print(f"Output directories created: {output_dir}")

# Display advanced streaming configurations
print("\nAdvanced Streaming Configuration:")
advanced_configs = [
    "spark.sql.adaptive.enabled",
    "spark.sql.streaming.checkpointLocation", 
    "spark.sql.streaming.stateStore.stateSchemaCheck",
    "spark.default.parallelism"
]

for config in advanced_configs:
    value = spark.conf.get(config, "Not Set")
    print("   {}: {}".format(config, value))

print("\nAdvanced Streaming environment ready!")
print("Ready for complex streaming patterns and stateful operations")

Setting up Advanced PySpark Streaming Environment...
Advanced Streaming Session Created
Spark Version: 4.0.0
Advanced checkpoint location: /tmp/advanced-streaming-checkpoints
Output directories created: /tmp/advanced_streaming_output

Advanced Streaming Configuration:
   spark.sql.adaptive.enabled: true
   spark.sql.streaming.checkpointLocation: /tmp/advanced-streaming-checkpoints
   spark.sql.streaming.stateStore.stateSchemaCheck: false
   spark.default.parallelism: 4

Advanced Streaming environment ready!
Ready for complex streaming patterns and stateful operations


In [2]:
# Sliding Window Analytics with Rate Source
print("Creating sliding window analytics with synthetic data...")

# 1. Create a rate source for testing - generates data at specified rate
print("=== 1. Rate Source Setup ===")

rate_stream = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 5) \
    .option("numPartitions", 2) \
    .load()

print("Rate source created - generating 5 rows per second")

# Add synthetic business data to the rate stream
enhanced_rate_stream = rate_stream \
    .withColumn("customer_id", (col("value") % 100).cast("string")) \
    .withColumn("transaction_amount", round(rand() * 1000, 2)) \
    .withColumn("product_category", 
        when(col("value") % 4 == 0, "Electronics")
        .when(col("value") % 4 == 1, "Clothing")
        .when(col("value") % 4 == 2, "Books")
        .otherwise("Sports")) \
    .withColumn("transaction_time", col("timestamp")) \
    .select("customer_id", "transaction_amount", "product_category", "transaction_time")

print("Enhanced rate stream with business data created")

# 2. Sliding Window Analytics - 30 second windows, sliding every 10 seconds
print("\n=== 2. Sliding Window Operations ===")

sliding_window_analytics = enhanced_rate_stream \
    .withWatermark("transaction_time", "1 minute") \
    .groupBy(
        window(col("transaction_time"), "30 seconds", "10 seconds"),  # 30s window, 10s slide
        col("product_category")
    ) \
    .agg(
        count("*").alias("transaction_count"),
        sum("transaction_amount").alias("total_sales"),
        avg("transaction_amount").alias("avg_transaction"),
        max("transaction_amount").alias("max_transaction"),
        approx_count_distinct("customer_id").alias("unique_customers")  # Use approx for streaming
    ) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        "product_category",
        "transaction_count", 
        "total_sales",
        "avg_transaction",
        "max_transaction",
        "unique_customers"
    )

# Start sliding window query
sliding_query = sliding_window_analytics \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .trigger(processingTime='5 seconds') \
    .start()

print("Sliding window analytics started (30s windows, 10s slide)")
print("Monitoring sales data across overlapping time periods...")

# Let it run to see overlapping windows
time.sleep(25)

print("\nSliding window demonstration complete!")
print("Notice how windows overlap - each transaction appears in multiple windows")

Creating sliding window analytics with synthetic data...
=== 1. Rate Source Setup ===
Rate source created - generating 5 rows per second
Rate source created - generating 5 rows per second
Enhanced rate stream with business data created

=== 2. Sliding Window Operations ===
Enhanced rate stream with business data created

=== 2. Sliding Window Operations ===
Sliding window analytics started (30s windows, 10s slide)
Monitoring sales data across overlapping time periods...
Sliding window analytics started (30s windows, 10s slide)
Monitoring sales data across overlapping time periods...
-------------------------------------------
Batch: 0
-------------------------------------------
+------------+----------+----------------+-----------------+-----------+---------------+---------------+----------------+
|window_start|window_end|product_category|transaction_count|total_sales|avg_transaction|max_transaction|unique_customers|
+------------+----------+----------------+-----------------+---------

In [3]:
# Stream-to-Stream Joins
print("Setting up stream-to-stream joins...")

# Stop previous query first
sliding_query.stop()
time.sleep(2)

print("=== 3. Stream-to-Stream Joins ===")

# Create two different streams to join
# Stream 1: Customer transactions (reuse enhanced rate stream)
transaction_stream = enhanced_rate_stream \
    .withColumn("join_key", col("customer_id")) \
    .withColumn("event_time", col("transaction_time")) \
    .select("join_key", "transaction_amount", "product_category", "event_time")

# Stream 2: Customer profile updates (another rate source with different data)
profile_stream = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 2) \
    .option("numPartitions", 1) \
    .load() \
    .withColumn("join_key", ((col("value") + 50) % 100).cast("string")) \
    .withColumn("customer_tier", 
        when(col("value") % 3 == 0, "Gold")
        .when(col("value") % 3 == 1, "Silver")
        .otherwise("Bronze")) \
    .withColumn("loyalty_points", (rand() * 10000).cast("int")) \
    .withColumn("event_time", col("timestamp")) \
    .select("join_key", "customer_tier", "loyalty_points", "event_time")

print("Created two streams for joining:")
print("- Transaction stream: customer purchases")  
print("- Profile stream: customer tier updates")

# Perform stream-to-stream join with time bounds
joined_stream = transaction_stream.alias("t") \
    .withWatermark("event_time", "1 minute") \
    .join(
        profile_stream.alias("p").withWatermark("event_time", "1 minute"),
        expr("""
            t.join_key = p.join_key AND
            t.event_time >= p.event_time - interval 30 seconds AND
            t.event_time <= p.event_time + interval 30 seconds
        """),
        "inner"
    ) \
    .select(
        col("t.join_key").alias("customer_id"),
        col("t.transaction_amount").alias("transaction_amount"),
        col("t.product_category").alias("product_category"), 
        col("p.customer_tier").alias("customer_tier"),
        col("p.loyalty_points").alias("loyalty_points"),
        col("t.event_time").alias("transaction_time"),
        col("p.event_time").alias("profile_update_time")
    )

# Start the join query
join_query = joined_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .trigger(processingTime='8 seconds') \
    .start()

print("Stream-to-stream join started!")
print("Joining transactions with customer profiles within 30-second time windows")

# Let it run to see joins
time.sleep(30)

print("\nStream join demonstration complete!")
print("Shows how to correlate data across multiple real-time streams")

Setting up stream-to-stream joins...
=== 3. Stream-to-Stream Joins ===
Created two streams for joining:
- Transaction stream: customer purchases
- Profile stream: customer tier updates
=== 3. Stream-to-Stream Joins ===
Created two streams for joining:
- Transaction stream: customer purchases
- Profile stream: customer tier updates
Stream-to-stream join started!
Joining transactions with customer profiles within 30-second time windows
Stream-to-stream join started!
Joining transactions with customer profiles within 30-second time windows
-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+------------------+----------------+-------------+--------------+----------------+-------------------+
|customer_id|transaction_amount|product_category|customer_tier|loyalty_points|transaction_time|profile_update_time|
+-----------+------------------+----------------+-------------+--------------+----------------+-------------------+
+-----------

In [6]:
# Socket Streaming and Memory Source
print("Setting up alternative data sources...")

# Stop previous query
join_query.stop()
time.sleep(2)

print("=== 4. Socket Streaming Source ===")

# Create a simple socket server to send data
def create_socket_server():
    """Create a socket server that sends sample data"""
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    
    try:
        server_socket.bind(('localhost', 9999))
        server_socket.listen(1)
        print("Socket server listening on localhost:9999")
        
        # Wait for client connection
        client_socket, addr = server_socket.accept()
        print(f"Client connected from {addr}")
        
        # Send sample log data
        log_messages = [
            "2025-08-26 10:30:01 INFO User login successful user_id=1001",
            "2025-08-26 10:30:05 ERROR Database connection failed retry_count=3", 
            "2025-08-26 10:30:08 INFO API request processed endpoint=/api/users duration=150ms",
            "2025-08-26 10:30:12 WARN High memory usage threshold=85%",
            "2025-08-26 10:30:15 INFO User logout user_id=1001 session_duration=900s",
            "2025-08-26 10:30:18 ERROR Payment processing failed transaction_id=tx_12345",
            "2025-08-26 10:30:22 INFO Cache invalidated cache_key=user_sessions",
            "2025-08-26 10:30:25 DEBUG Query executed query_time=45ms table=products"
        ]
        
        for i, message in enumerate(log_messages):
            time.sleep(3)  # Send message every 3 seconds
            client_socket.send((message + '\n').encode())
            print(f"Sent log message {i+1}")
            
        time.sleep(5)
        client_socket.close()
        
    except Exception as e:
        print(f"Socket server error: {e}")
    finally:
        server_socket.close()

# Start socket server in background thread
socket_thread = threading.Thread(target=create_socket_server, daemon=True)
socket_thread.start()

# Give server time to start
time.sleep(1)

# Create socket stream (note: this may fail if connection issues)
try:
    socket_stream = spark \
        .readStream \
        .format("socket") \
        .option("host", "localhost") \
        .option("port", 9999) \
        .load()
    
    # Parse log data
    parsed_logs = socket_stream \
        .withColumn("log_level", regexp_extract(col("value"), r"(INFO|ERROR|WARN|DEBUG)", 1)) \
        .withColumn("timestamp", regexp_extract(col("value"), r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})", 1)) \
        .withColumn("message", col("value")) \
        .filter(col("log_level") != "")
    
    # Start socket streaming query
    socket_query = parsed_logs \
        .writeStream \
        .outputMode("append") \
        .format("console") \
        .option("truncate", False) \
        .trigger(processingTime='5 seconds') \
        .start()
    
    print("Socket streaming started - processing log data")
    time.sleep(30)
    socket_query.stop()
    
except Exception as e:
    print(f"Socket streaming not available (expected in some environments): {e}")

print("\n=== 5. Memory Source for Testing ===")

# Create memory source for testing and debugging
memory_data = [
    {"id": 1, "name": "Alice", "score": 95, "timestamp": datetime.now()},
    {"id": 2, "name": "Bob", "score": 87, "timestamp": datetime.now()},
    {"id": 3, "name": "Charlie", "score": 92, "timestamp": datetime.now()}
]

# Convert to DataFrame and create temporary view for memory demo
df = spark.createDataFrame(memory_data, ["test_id", "value", "timestamp"])
df.createOrReplaceTempView("test_data")

# Read from the temp view using SQL (simulates memory table reading)
print("Created temporary table 'test_data' with sample data")
spark.sql("SELECT * FROM test_data").show()

# Alternative approach: Use rate source to demonstrate memory sink pattern
print("\n=== Alternative Memory Pattern with Rate Source ===")
memory_test_stream = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 2) \
    .load() \
    .withColumn("test_id", concat(lit("test_"), col("value").cast("string"))) \
    .withColumn("test_value", rand() * 100) \
    .select("test_id", "test_value", "timestamp")

print("Created rate-based stream for memory sink demonstration")

# Write to memory sink to create queryable table
memory_sink_query = memory_test_stream \
    .writeStream \
    .format("memory") \
    .queryName("memory_demo_table") \
    .outputMode("append") \
    .start()

print("Memory sink started - data available in 'memory_demo_table'")

# Let it accumulate some data
time.sleep(8)

# Query the memory table
print("Sample data from memory sink:")
spark.sql("SELECT COUNT(*) as total_records FROM memory_demo_table").show()
spark.sql("SELECT * FROM memory_demo_table ORDER BY timestamp DESC LIMIT 5").show()

# Stop the memory query
memory_sink_query.stop()
print("Memory source demonstration complete!")
print("Useful for testing and development with known datasets")

Setting up alternative data sources...
=== 4. Socket Streaming Source ===
Socket server listening on localhost:9999
=== 4. Socket Streaming Source ===
Socket server listening on localhost:9999
Socket streaming started - processing log data
Client connected from ('127.0.0.1', 54966)
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---------+---------+-------+
|value|log_level|timestamp|message|
+-----+---------+---------+-------+
+-----+---------+---------+-------+

Socket streaming started - processing log data
Client connected from ('127.0.0.1', 54966)
-------------------------------------------
Batch: 0
-------------------------------------------
+-----+---------+---------+-------+
|value|log_level|timestamp|message|
+-----+---------+---------+-------+
+-----+---------+---------+-------+

Sent log message 1
Sent log message 1
Sent log message 2
Sent log message 2
-------------------------------------------
Batch: 1
--------------

In [7]:
# Stateful Stream Processing
print("Setting up stateful stream processing...")

print("=== 6. Stateful Processing with mapGroupsWithState ===")

# Create a simple stream for stateful demo using rate source
stateful_input = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 3) \
    .load() \
    .withColumn("user_id", (col("value") % 10).cast("string")) \
    .withColumn("action", 
        when(col("value") % 3 == 0, "login")
        .when(col("value") % 3 == 1, "purchase") 
        .otherwise("logout")) \
    .withColumn("event_time", col("timestamp")) \
    .select("user_id", "action", "event_time")

print("Created input stream for stateful processing")

# Define state structure for user sessions
from pyspark.sql.streaming.state import GroupState, GroupStateTimeout

# User session state schema
session_state_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("session_start", TimestampType(), True),
    StructField("last_activity", TimestampType(), True),
    StructField("action_count", IntegerType(), True),
    StructField("actions", ArrayType(StringType()), True)
])

# Output schema for session updates  
session_output_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("session_duration_minutes", DoubleType(), True),
    StructField("total_actions", IntegerType(), True),
    StructField("session_status", StringType(), True),
    StructField("last_action", StringType(), True)
])

def update_user_session(user_id, events, state):
    """
    Stateful function to track user sessions
    """
    from datetime import datetime
    
    # Get current state or initialize
    if state.exists:
        session = state.get
        session_start = session.session_start
        action_count = session.action_count
        actions = list(session.actions) if session.actions else []
    else:
        # New session
        session_start = None
        action_count = 0
        actions = []
    
    # Process new events
    for event in events:
        if session_start is None:
            session_start = event.event_time
            
        action_count += 1
        actions.append(event.action)
        last_activity = event.event_time
    
    # Update state
    if session_start:
        # Calculate session duration in minutes
        duration = (last_activity - session_start).total_seconds() / 60.0 if last_activity > session_start else 0.0
        
        # Determine session status
        last_action = actions[-1] if actions else "unknown"
        session_status = "active" if last_action != "logout" else "ended"
        
        # Create new state
        new_state = spark.createDataFrame([{
            "user_id": user_id,
            "session_start": session_start,
            "last_activity": last_activity,
            "action_count": action_count,
            "actions": actions
        }], session_state_schema).collect()[0]
        
        state.update(new_state)
        
        # Set timeout for inactive sessions (5 minutes)
        if session_status == "active":
            state.setTimeoutDuration("5 minutes")
        
        # Return session summary
        return [{
            "user_id": user_id,
            "session_duration_minutes": duration,
            "total_actions": action_count,
            "session_status": session_status,
            "last_action": last_action
        }]
    
    return []

# Note: mapGroupsWithState requires Scala/Java implementation in practice
# For demonstration, we'll use a simpler stateful aggregation

print("Using simplified stateful aggregation (mapGroupsWithState requires Scala)")

# Simplified stateful processing using built-in aggregations
stateful_sessions = stateful_input \
    .withWatermark("event_time", "10 minutes") \
    .groupBy("user_id") \
    .agg(
        min("event_time").alias("session_start"),
        max("event_time").alias("last_activity"),
        count("*").alias("total_actions"),
        collect_list("action").alias("actions"),
        last("action").alias("last_action")
    ) \
    .withColumn("session_duration_minutes", 
        (unix_timestamp("last_activity") - unix_timestamp("session_start")) / 60.0) \
    .withColumn("session_status",
        when(col("last_action") == "logout", "ended").otherwise("active"))

# Start stateful query
stateful_query = stateful_sessions \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", False) \
    .trigger(processingTime='10 seconds') \
    .start()

print("Stateful session tracking started!")
print("Tracking user sessions with state accumulation")

time.sleep(35)
stateful_query.stop()

print("\nStateful processing demonstration complete!")
print("Shows how to maintain state across streaming batches")

Setting up stateful stream processing...
=== 6. Stateful Processing with mapGroupsWithState ===
Created input stream for stateful processing
Using simplified stateful aggregation (mapGroupsWithState requires Scala)
Stateful session tracking started!
Tracking user sessions with state accumulation
-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-------------+-------------+-------------+-------+-----------+------------------------+--------------+
|user_id|session_start|last_activity|total_actions|actions|last_action|session_duration_minutes|session_status|
+-------+-------------+-------------+-------------+-------+-----------+------------------------+--------------+
+-------+-------------+-------------+-------------+-------+-----------+------------------------+--------------+

-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-------------+-------------+-------------+-------+-

In [8]:
# Advanced Output Sinks
print("Setting up advanced output sinks...")

print("=== 7. File Output Sinks ===")

# Create a clean rate stream for output demonstrations
output_stream = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", 4) \
    .load() \
    .withColumn("category", 
        when(col("value") % 4 == 0, "A")
        .when(col("value") % 4 == 1, "B") 
        .when(col("value") % 4 == 2, "C")
        .otherwise("D")) \
    .withColumn("amount", round(rand() * 100, 2)) \
    .withColumn("event_date", to_date(col("timestamp"))) \
    .select("timestamp", "value", "category", "amount", "event_date")

print("Created output stream for sink demonstrations")

# 1. Parquet file sink with partitioning
print("\n--- Parquet File Sink ---")
parquet_query = output_stream \
    .writeStream \
    .format("parquet") \
    .option("path", parquet_dir) \
    .option("checkpointLocation", "/tmp/advanced-streaming-checkpoints/parquet") \
    .partitionBy("category") \
    .trigger(processingTime='10 seconds') \
    .start()

print(f"Parquet sink started, writing to: {parquet_dir}")

# 2. JSON file sink with custom options
print("\n--- JSON File Sink ---")
json_query = output_stream \
    .select("timestamp", "value", "category", "amount") \
    .writeStream \
    .format("json") \
    .option("path", json_dir) \
    .option("checkpointLocation", "/tmp/advanced-streaming-checkpoints/json") \
    .trigger(processingTime='8 seconds') \
    .start()

print(f"JSON sink started, writing to: {json_dir}")

# Let sinks run to generate files
print("\nLet sinks run for 25 seconds to generate output files...")
time.sleep(25)

# Stop file queries
parquet_query.stop()
json_query.stop()

print("\n=== 8. Custom ForeachBatch Sink ===")

# Custom processing function for foreach batch
def process_batch(batch_df, batch_id):
    """Custom batch processing function"""
    print(f"\n--- Processing Batch {batch_id} ---")
    print(f"Batch size: {batch_df.count()} records")
    
    # Custom aggregation
    category_summary = batch_df.groupBy("category").agg(
        count("*").alias("count"),
        avg("amount").alias("avg_amount")
    ).collect()
    
    print("Category Summary:")
    for row in category_summary:
        print(f"  {row.category}: {row.count} records, avg amount: ${row.avg_amount:.2f}")
    
    # Could write to database, send alerts, etc.
    print("Custom processing complete for this batch")

# Create foreach batch query
foreach_query = output_stream \
    .writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime='12 seconds') \
    .start()

print("ForeachBatch custom sink started!")
print("Processing batches with custom logic...")

time.sleep(30)
foreach_query.stop()

print("\n=== 9. Memory Sink for Testing ===")

# Memory sink stores results in memory table for inspection
memory_sink_query = output_stream \
    .select("category", "amount") \
    .groupBy("category") \
    .agg(
        count("*").alias("total_records"),
        sum("amount").alias("total_amount"),
        avg("amount").alias("avg_amount")
    ) \
    .writeStream \
    .format("memory") \
    .queryName("streaming_results") \
    .outputMode("update") \
    .trigger(processingTime='5 seconds') \
    .start()

print("Memory sink started - storing results in 'streaming_results' table")

# Let it accumulate some data
time.sleep(15)

# Query the memory table
print("\nQuerying accumulated results from memory sink:")
results = spark.sql("SELECT * FROM streaming_results ORDER BY category")
results.show()

memory_sink_query.stop()

# Check what files were created
print(f"\n=== Output Files Created ===")
print(f"Parquet files in {parquet_dir}:")
import subprocess
try:
    result = subprocess.run(['find', parquet_dir, '-name', '*.parquet'], capture_output=True, text=True)
    print(result.stdout)
except:
    print("Could not list parquet files")

print(f"\nJSON files in {json_dir}:")
try:
    result = subprocess.run(['find', json_dir, '-name', '*.json'], capture_output=True, text=True)
    print(result.stdout)
except:
    print("Could not list JSON files")

print("\nAdvanced output sinks demonstration complete!")
print("Demonstrated file sinks, custom processing, and memory storage")

Setting up advanced output sinks...
=== 7. File Output Sinks ===
Created output stream for sink demonstrations

--- Parquet File Sink ---
Parquet sink started, writing to: /tmp/advanced_streaming_output/parquet_sink

--- JSON File Sink ---
JSON sink started, writing to: /tmp/advanced_streaming_output/json_sink

Let sinks run for 25 seconds to generate output files...
JSON sink started, writing to: /tmp/advanced_streaming_output/json_sink

Let sinks run for 25 seconds to generate output files...



=== 8. Custom ForeachBatch Sink ===
ForeachBatch custom sink started!
Processing batches with custom logic...

--- Processing Batch 0 ---
Batch size: 0 records
Category Summary:
Custom processing complete for this batch

--- Processing Batch 0 ---
Batch size: 0 records
Category Summary:
Custom processing complete for this batch

--- Processing Batch 1 ---
Batch size: 40 records
Category Summary:
  B: <built-in method count of Row object at 0x11b5ea9f0> records, avg amount: $62.26
  C: <built-in method count of Row object at 0x11654f9f0> records, avg amount: $52.20
  A: <built-in method count of Row object at 0x11c6fae00> records, avg amount: $47.80
  D: <built-in method count of Row object at 0x11c76b3b0> records, avg amount: $59.45
Custom processing complete for this batch

--- Processing Batch 1 ---
Batch size: 40 records
Category Summary:
  B: <built-in method count of Row object at 0x11b5ea9f0> records, avg amount: $62.26
  C: <built-in method count of Row object at 0x11654f9f0> r

In [9]:
# Module 7B Summary and Cleanup
print("=== Module 7B: Advanced Streaming Patterns - Complete! ===")

# Stop any remaining active queries
print("\nCleaning up remaining streaming queries...")
for stream in spark.streams.active:
    print(f"Stopping: {stream.name if stream.name else 'Unnamed Query'}")
    stream.stop()

print("All streaming queries stopped")

# Summary of advanced patterns covered
print("\n" + "="*70)
print("MODULE 7B ADVANCED ACCOMPLISHMENTS")
print("="*70)

print("\n✅ ADVANCED WINDOWING PATTERNS")
print("   • Sliding windows with overlapping time periods")
print("   • 30-second windows sliding every 10 seconds")
print("   • Multi-dimensional aggregations across time")

print("\n✅ STREAM JOIN OPERATIONS")
print("   • Stream-to-stream joins with time bounds")
print("   • Real-time correlation of transaction and profile data")
print("   • Temporal join conditions with watermarks")

print("\n✅ ALTERNATIVE DATA SOURCES")
print("   • Rate sources for testing and benchmarking")
print("   • Socket sources for real-time TCP data")
print("   • Memory sources for development and debugging")

print("\n✅ STATEFUL STREAM PROCESSING")
print("   • Session tracking with state management")
print("   • User activity aggregation across time")
print("   • Complex stateful transformations")

print("\n✅ ADVANCED OUTPUT PATTERNS")
print("   • Partitioned Parquet file sinks")
print("   • JSON file outputs with custom triggers")
print("   • ForeachBatch custom processing")
print("   • Memory sinks for testing and validation")

print("\n" + "="*70)
print("ADVANCED STREAMING METRICS")
print("="*70)

print(f"\n🔄 Streaming Patterns:")
print(f"   • Sliding window analytics with overlap detection")
print(f"   • Multi-stream joins with temporal constraints")
print(f"   • Alternative source integrations")

print(f"\n🏗️ Architecture Patterns:")
print(f"   • Stateful processing for session management")
print(f"   • Custom output sinks for flexible integration")
print(f"   • Memory-based testing and validation")

print(f"\n⚙️ Production Features:")
print(f"   • Partitioned file outputs for scalability")
print(f"   • Custom batch processing logic")
print(f"   • Multiple concurrent streaming patterns")

print("\n" + "="*70)
print("STREAMING EXPERTISE ACHIEVED")
print("="*70)

print("\n🎓 Core + Advanced Streaming Mastery:")
print("   ✅ Module 7: File streaming, basic windowing, IoT analytics")
print("   ✅ Module 7B: Sliding windows, joins, stateful processing")
print("   ✅ Production-ready streaming architecture patterns")

print("\n🚀 Ready for Enterprise Streaming Applications:")
print("   • Real-time analytics dashboards")
print("   • Complex event processing systems")
print("   • Multi-source data correlation")
print("   • Stateful business logic implementation")

# Clean up temporary directories
import shutil
try:
    shutil.rmtree("/tmp/advanced_streaming_output", ignore_errors=True)
    shutil.rmtree("/tmp/advanced-streaming-checkpoints", ignore_errors=True)
    print("\n🧹 Advanced streaming artifacts cleaned up")
except:
    pass

print("\n" + "="*70)
print("🎯 ADVANCED STREAMING PATTERNS MASTERY COMPLETE!")
print("Ready for Module 8: ML Integration or Graph Analytics!")
print("="*70)

=== Module 7B: Advanced Streaming Patterns - Complete! ===

Cleaning up remaining streaming queries...
All streaming queries stopped

MODULE 7B ADVANCED ACCOMPLISHMENTS

✅ ADVANCED WINDOWING PATTERNS
   • Sliding windows with overlapping time periods
   • 30-second windows sliding every 10 seconds
   • Multi-dimensional aggregations across time

✅ STREAM JOIN OPERATIONS
   • Stream-to-stream joins with time bounds
   • Real-time correlation of transaction and profile data
   • Temporal join conditions with watermarks

✅ ALTERNATIVE DATA SOURCES
   • Rate sources for testing and benchmarking
   • Socket sources for real-time TCP data
   • Memory sources for development and debugging

✅ STATEFUL STREAM PROCESSING
   • Session tracking with state management
   • User activity aggregation across time
   • Complex stateful transformations

✅ ADVANCED OUTPUT PATTERNS
   • Partitioned Parquet file sinks
   • JSON file outputs with custom triggers
   • ForeachBatch custom processing
   • Memor